In [1]:
import cv2
import numpy as np
import os

In [2]:
class_names = os.listdir('Train')
class_names

['Bikes', 'Horses']

In [3]:
images = [] #Get path to all images and save them in a list
labels = [] # labels of corresponding images
class_id = 0

for i in class_names:
    dir=os.path.join('train',i)
    x = [os.path.join(dir, f) for f in os.listdir(dir)]
    images+= x
    labels+=[class_id]*len(x)
    class_id+=1

In [4]:
import random
D = []
for i in range(len(images)):
    D.append([images[i],labels[i]])

dataset = D
random.shuffle(dataset)
train = dataset[:155]
test = dataset[155:]

In [16]:
train

[['train\\Bikes\\0078.jpg', 0],
 ['train\\Bikes\\0064.jpg', 0],
 ['train\\Horses\\horse1  (33).jpg', 1],
 ['train\\Horses\\horse51.jpg', 1],
 ['train\\Horses\\horse1  (31).jpg', 1],
 ['train\\Horses\\horse1  (37).jpg', 1],
 ['train\\Bikes\\0058.jpg', 0],
 ['train\\Horses\\horse42.jpg', 1],
 ['train\\Bikes\\0020.jpg', 0],
 ['train\\Horses\\horse44.jpg', 1],
 ['train\\Bikes\\0044.jpg', 0],
 ['train\\Horses\\horse64.jpg', 1],
 ['train\\Bikes\\0042.jpg', 0],
 ['train\\Bikes\\0069.jpg', 0],
 ['train\\Horses\\horse11.jpg', 1],
 ['train\\Horses\\horse71.jpg', 1],
 ['train\\Horses\\horse22.jpg', 1],
 ['train\\Bikes\\0063.jpg', 0],
 ['train\\Horses\\horse23.jpg', 1],
 ['train\\Horses\\horse1  (17).jpg', 1],
 ['train\\Horses\\horse63.jpg', 1],
 ['train\\Horses\\horse1 .jpg', 1],
 ['train\\Horses\\horse70.jpg', 1],
 ['train\\Horses\\horse7 .jpg', 1],
 ['train\\Horses\\horse43.jpg', 1],
 ['train\\Horses\\horse2 .jpg', 1],
 ['train\\Bikes\\0043.jpg', 0],
 ['train\\Bikes\\0018.jpg', 0],
 ['train\\Ho

In [5]:
!pip install torch

     -------------------------------------- 172.3/172.3 MB 6.3 MB/s eta 0:00:00


In [6]:
!pip install torch-summary

In [8]:
!pip install torchvision

     ---------------------------------------- 1.2/1.2 MB 4.2 MB/s eta 0:00:00


In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torchsummary import summary

In [11]:
features = models.alexnet().features  # get the feature extraction layers
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
flatten = torch.nn.Flatten()# create a flatten layer
model = torch.nn.Sequential(features, flatten).to(device)  # create a new model with the flatten layer
summary(model)

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─Flatten: 1-2                           --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─Flatten: 1-2                           --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0

In [40]:
def extract_features(train):
    features = []
    labels = []
    for i in train:
        im = cv2.imread(i[0])
        im = im.astype('float32')
        im = cv2.resize(im, (224,224))
        im = torch.from_numpy(im)
        im = im.unsqueeze(0) # adding an extra dimension at index 0 which refers to the batch size =1
        im = im.permute(0, 3, 1, 2) # reorder the dimensions of the input tensor from [batch_size, height, width, num_channels] to [batch_size, num_channels, height, width]. 
        with torch.no_grad():
            outputs = model(im)
            features.append(outputs.numpy()[0])
            labels.append(i[1])
    return np.array(features), np.array(labels)

In [41]:
x_train, y_train = extract_features(train)
x_test, y_test = extract_features(test)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(155, 9216)
(155,)
(24, 9216)
(24,)


In [44]:
from sklearn import svm
clf = svm.SVC(C=1.8, random_state=42,verbose =2)
clf.fit(x_train, y_train)

[LibSVM]

SVC(C=1.8, random_state=42, verbose=2)

In [45]:
from sklearn.metrics import accuracy_score
pred_labels = clf.predict(x_test)
accuracy = accuracy_score(y_test, pred_labels)
print('Accuracy:', accuracy)

Accuracy: 1.0


In [46]:
pred_labels

array([0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0])

In [47]:
y_test

array([0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0])

In [48]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, pred_labels)
print(cm)

[[ 9  0]
 [ 0 15]]


In [49]:
from sklearn.metrics import f1_score

# y_true is the true labels and y_pred is the predicted labels
f1score = f1_score(y_test, pred_labels, average='weighted')

print("F1 score: {:.2f}".format(f1score))

F1 score: 1.00
